In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv("../input/san-francisco-crime-classification/train.csv", parse_dates=['Dates'])
train.head()

In [ ]:
train.dtypes

In [ ]:
train.duplicated().sum()
train.drop_duplicates(keep='first',inplace=True)

In [ ]:
train.shape

In [ ]:
target =  train['Category'].unique()
print(target)

In [ ]:
len(target)

In [ ]:
test = pd.read_csv("../input/san-francisco-crime-classification/test.csv", parse_dates=['Dates'])

In [ ]:
test.head()


In [ ]:
cat_dict = {}
count = 1
for data in target:
    cat_dict[data] = count
    count+=1
train['Category'] = train['Category'].replace(cat_dict)

week_dict = { "Sunday" : 1, "Monday":2, "Tuesday": 3, "Wednesday":4, "Thursday":5, "Friday":6, "Saturday":7}
train['DayOfWeek'] = train['DayOfWeek'].replace(week_dict)
test['DayOfWeek'] = test['DayOfWeek'].replace(week_dict)

In [ ]:
train.head()

In [ ]:
district = train['PdDistrict'].unique()
print(district)

In [ ]:
district_dict = {}
count = 1
for data in district:
    district_dict[data]= count
    count+=1
train['PdDistrict'] = train['PdDistrict'].replace(district_dict)
test['PdDistrict'] = test['PdDistrict'].replace(district_dict)
train.head()

In [ ]:
col1 = train.columns
col1

In [ ]:
col2 =  test.columns
col2

In [ ]:
new_col1 = col1.drop("Resolution")
new_train = train[new_col1]


In [ ]:
new_train.describe()

In [ ]:
correlation = new_train.corr()
print(correlation['Category'])

In [ ]:
skew = new_train.skew()
print(skew)

In [ ]:
# variables = ['DayOfWeek', 'PdDistrict','X','Y']
# X_train = train[variables]
# Y_train = train['Category']
# X_test = test[variables]

# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=5)
# knn.fit(X_train, Y_train)
# prediction = knn.predict(X_test)

In [ ]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train[['DayOfWeek', 'PdDistrict','X','Y']])


In [ ]:
scaled_features = scaler.transform(train[['DayOfWeek', 'PdDistrict','X','Y']])


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaled_features, new_train['Category'], test_size=0.2)

In [ ]:
# variables = ['DayOfWeek', 'PdDistrict','X','Y']
# X_train = train[variables]
# Y_train = train['Category']
# X_test = test[variables]

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
prediction = knn.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report , confusion_matrix
from sklearn.model_selection import cross_val_score

In [ ]:
print(confusion_matrix(y_test,prediction))

In [ ]:
print(classification_report(y_test,prediction))

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.model_selection import cross_val_score
# accuracy_r =[]
# for i in range(1,40):
#     knn = KNeighborsClassifier(n_neighbors=i)
#     score = cross_val_score(knn,train[['DayOfWeek', 'PdDistrict','X','Y']], train['Category'], cv=6)
#     accuracy_r.append(score.mean())

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline

# plt.figure(figsize=(10,6))
# #plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
#   #       markerfacecolor='red', markersize=10)
# plt.plot(range(1,40),accuracy_r,color='blue', linestyle='dashed', marker='o',
#          markerfacecolor='red', markersize=10)

In [ ]:
from collections import OrderedDict
new_cat_dict = OrderedDict(sorted(cat_dict.items()))

new_cat_dict

In [ ]:
result = pd.DataFrame({
    "Id": test["Id"]
})
for key,value in new_cat_dict.items():
    result[key] = 0
count = 0
for data in prediction:
    for key,value in new_cat_dict.items():
        if(data==value):
            result[key][count] =1
    count+=1

In [ ]:
# from sklearn import preprocessing
# label_encoder = preprocessing.LabelEncoder()
# for var in target:
#     train[var]= label_encoder.fit_transform(train[var])
# result = label_encoder.inverse_transform(prediction)
result.to_csv("predictions.csv", index=False)